# Programming Assignment 2

In [ ]:
#Please make sure you are using a python3.9.x interpreter 
# and install the packages from requirements.txt (provided)
!pip install -r requirements.txt

In [1]:
#Some basic imports. You can import any package at any cell
from sklearn.preprocessing import OneHotEncoder   #My favorite categorical to numerical feature conversion tool
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from numpy.linalg import inv
from time import perf_counter
import math
import sys
import random

In [2]:
### Let's load the given dataset
# First load the dataset into pandas dataframe
full_dataset = pd.read_csv('dataset/baby-weights-dataset.csv',delimiter=',')
judge_dataset = pd.read_csv('dataset/judge-without-labels.csv',delimiter=',')

TASK 1:
Separate the full_dataset into two parts: X and y, where X denotes the input matrix containing only the input (i.e., independent explanatory) variables, and y denotes the target variable containing only the target values for exactly the same number of samples in the given full_dataset.

In [3]:
#Make sure you write your solution to this task below in this cell only.

# YOUR CODE HERE

X = full_dataset.iloc[:,:-1]
y = full_dataset.iloc[:,-1]
print(X.shape)
print(y.shape)

(101400, 36)
(101400,)


In [4]:
#Let me test your code above
assert X.shape==(101400,36)
assert y.shape==(101400,)

TASK 2:
Given X representing the input matrix from the full_dataset, y being the target vector (the rightmost column of the full_dataset), obtained from Task 1:
randomly split the (X,y) dataset into 75% for training and 25% for testing using the library function from the library sklearn.model_selection . Please pass to the train_test_split function an additional argument random_state=45931.
Store the 4 splits as X_train, X_test, y_train, y_test respectively.
Save the ID column for X_train and X_test into ID_train and ID_test as list variable.
Now, drop the ID columns from both X_train and X_test

In [5]:
#Make sure you write your solution to this task below in this cell only.

X_train = []
X_test = []
y_train = []
y_test = []

# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=45931)

ID_train = X_train["ID"]
ID_test = X_test["ID"]

X_train = X_train.drop("ID", axis=1)
X_test = X_test.drop("ID", axis=1)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(76050, 35) (25350, 35) (76050,) (25350,)


In [6]:
#Let me test your code above
assert (X_train.shape, X_test.shape, y_train.shape, y_test.shape) == ((76050, 35), (25350, 35), (76050,), (25350,))

TASK 3:
Compute mean, stdev, min, max, 25% percentile, median and 75% percentile of BWEIGHT target variable (i.e, the target y) in the training set (i.e., y_train), and print the computed values as a numpy array containing these 7 results (respectively).

In [7]:
#Make sure you write your solution to this task below in this cell only.
mean_val = 0
stdev_val = 0
min_val = 0
max_val = 0
percentile25_val = 0
median_val = 0
percentile75_val = 0


# YOUR CODE HERE



mean_val = np.mean(y_train)
stdev_val = np.std(y_train)
min_val = np.min(y_train)
max_val = np.max(y_train)
percentile25_val = np.percentile(y_train, 25)
median_val = np.percentile(y_train, 50)
percentile75_val = np.percentile(y_train, 75)

print(mean_val,stdev_val,min_val,max_val,percentile25_val,median_val,percentile75_val)


7.256997863247864 1.3300496889691478 0.3125 13.0625 6.625 7.375 8.0625


In [8]:
assert abs(sum([mean_val,stdev_val, min_val,max_val, percentile25_val,median_val,percentile75_val])-44.024547552217015) < 1e-4

TASK 4:
A little background first: Categorical features are features that contain values that are not numeric. As you can imagine non-numeric values will create trouble (by introducing nan) during calculation to gradients, and whatnot, right? The maths are undefined when these get in its way. An obvious solution you may be intrigued to do is dropping the features! Aha! Wrong!! Every piece of data is precious... as those non-numeric features may present with valuable insights of the data samples to find the patterns to map inputs with output/targets. So, we should include them. But, how?

The answer is via "Encoding" we can make good use the non-numeric variables.

There are several types of encoding used in practice. Here are the two popular ones:

Label Encoding, where labels are encoded as subsequent numbers. Say, for a categorical feature named "Category" with three categorical values: {“Cat”, “Dog” or “Zebra”} can be encoded to "0", "1", "2" respectively as in figure below. The issue with this type of encoding may unintentionally impose a type of ordering of the categories, that may add bias to the training.label-encoding
One Hot Encoding, ignores the ordering of the categories all together. With one-hot, we convert each categorical value into a new categorical column and assign a binary value of 1 or 0 to those columns. Each integer value is represented as a binary vector. All the values are zero, and the index is marked with a 1. Also, don't forget to remove the original categorical features. Here below just an example, how to convert the categorical feature called "Category" having the {“Cat”, “Dog” or “Zebra”} values into three new binary features: "Cat", "Dog", "Zebra".label-encoding
A note on the Dummy Variable Trap The Dummy Variable Trap occurs when two or more dummy variables created by one-hot encoding are highly correlated (i.e., becomes multi-collinear). This means that one variable can be predicted from the others, making it difficult to interpret predicted coefficient variables in regression models. In other words, the individual effect of the dummy variables on the prediction model can not be interpreted well because of multicollinearity.

Using the one-hot encoding method, a new dummy variable is created for each categorical variable to represent the presence (1) or absence (0) of the categorical variable. For example, if tree species is a categorical variable made up of the values pine, or oak, then tree species can be represented as a dummy variable by converting each variable to a one-hot vector. This means that a separate column is obtained for each category, where the first column represents if the tree is pine and the second column represents if the tree is oak. Each column will contain a 0 or 1 if the tree in question is of the column's species. These two columns are multi-collinear since if a tree is pine, then we know it's not oak and vice versa. The machine learning models trained on dataset having this multi-collinearity suffers. A remedy is to drop first (or any one) of the dummy (i.e., one-hot) features created.

Given the training dataset (X_train, y_train), save as X_train_ohe after replacing all the non-numeric variables (i.e., categorical variables) with numeric encoding. Also, you need to use the same encoding to work on the X_train dataset as well and save it as X_test_oh. We need to make sure any new categorical value in the test dataset get ignored, i.e, when an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros. Please complete the following almost complete function lets_do_one_hot_encoding() that encodes dataset on the supplied categorical feature column list.

In [9]:
#Make sure you write your solution to this task below in the marked location in this cell only.


def lets_do_one_hot_encoding(data, categorical_features, transform_only=True, encoders=[], verbose=False):
    '''
    The function does one_hot_encoding on the given dataset... 
    It's intentionally defined fully, meaning you do not need to do anything here... but show us
    how to use it properly.
    
    Input: 
        * data -- it's pretty much either X_train, or X_test that you prepared previously, that is
                  any dataframe having independent variables.
        * categorical_features -- a list of column/feature names in the data (dataframe) that you think
                  are non-numerical / i.e., categorical that you want to be converted to numerical
                  using the one-hot encoding technique.
        * transform_only -- a boolean parameter, if set to False, will learn (i.e., fit) various categorical
                values in the categorical_features from the given dataset, and use this to encode the dataset
                using one-hot encoding. This is important that you set to False on training dataset, and
                True on test set. If new categorical values are present in the test dataset, those will be
                ignored, making it easier to have same set of encoded features both in training and test 
                dataset... otherwise, subsequent operations (may/) will not work.
        * encoders -- a list of one-hot encoders previously saved, and now will be used to encode given dataset.
                If transform_only=True, the function looks for this provided list of encoders to encode the 
                dataset instead of learning new categories. Once again, it's expected that you pass the set
                of encoders for each of the categorical features that you encoded the training set -- i.e.,
                leave it empty for training set, and pass the set of encoders while encoding test set.
    Returns:
        * data -- the converted dataframe after the encoding is completed.
        * enc_list -- is the list of encoders used to encode the given data.
        * categorical_features -- is the list of categorical features that you would want to encode.
        It's expected that for training dataset, you save the encoder list and the list of features for later
        use to encode a test dataset.
    '''
    if len(encoders)>0:
        enc_list=encoders
    else:
        enc_list = []
    col_onehot = []
    i = 0
    for feature in categorical_features:
        if verbose: print('Dealing with feature ={}'.format(feature))
        if transform_only==True: #for test dataset
            enc = enc_list[i]
            c_onehot = enc.transform(data[[feature]])
            i = i + 1
        else: #fit and transform
            enc = OneHotEncoder(handle_unknown='ignore', drop='first', sparse=False)
            c_onehot = enc.fit_transform(data[[feature]])
            if verbose: print('c_onehot = {}'.format(c_onehot))
            enc_list.append(enc)
        c_onehot = pd.DataFrame(c_onehot, columns=list(enc.categories_[0][1:])) #dropped first column
        if verbose: print('dropped first column...c_onehot = {}'.format(c_onehot))
        c_onehot = c_onehot.add_prefix(feature+'_')
        col_onehot.append(c_onehot)
        if verbose: print('col_onehot = {}'.format(col_onehot))
    
    #concat all onehot feature columns
    concat_df = pd.concat(col_onehot,axis=1)
    #match index with given data
    concat_df.index = data.index
    
    #Here below is your task:
    #i) drop the categorical feature columns from dataframe `data`
    #ii) and then merge with those new onehot features stored in `cocat_df`
    # YOUR CODE HERE
    
    data = data.drop(categorical_features, axis=1)
    data = data.join(concat_df)
    
    return data,enc_list,categorical_features

In [10]:
'''
let's apply one-hot encoding on two columns: "HISPMOM" and "HISPDAD" of the training dataset, X_train.
Please call the function, lets_do_one_hot_encoding() you helped defining above passing appropriate arguments.
Also, receive the 3 return values from the function call: X_train_ohe, enc_list and categorical_features to 
use them in the next question, that is, encoding the test dataset, X_test, and 
later in Task 17 to encode the judge set. 

It's always a good idea to save the list of encoders and features in a file (e.g., joblib), if you would 
want to apply your model to evaluate/predict on a new test sample. In a summary: if your model was trained with
transformed one-hot encoded dataset, a new test data also needs to be encoded with the same list of variables
defined by the encoding.

'''
X_train_ohe = []  #one-hot encoded training dataset
enc_list = [] #encoder list to be created during one-hot encoding of the training dataset
categorical_features = ['HISPMOM', 'HISPDAD'] #The list of categorical features in question.

#Here below is your task:one-hot encoding of train set: X_train
# YOUR CODE HERE
X_train_ohe, enc_list, categorical_features = lets_do_one_hot_encoding(X_train, categorical_features, transform_only=False, encoders=[] )



In [11]:
'''
let's apply one-hot encoding on the same two columns: "HISPMOM" and "HISPDAD" of the test dataset, X_test.
Please call the function, lets_do_one_hot_encoding() you helped defining above with appropriate parameters.
Be sure to use the encoders from the training set and turn on the `transform_only` switch to true.
Also, receive the 3 return values from the function call in X_test_ohe, enc_list, categorical_features. 
Although, the second and third returned values can be ignored.

Please make sure, the one-hot encoded training set and test set has the exact same number of features/columns,
and in the same order. If not, the test in the next cell will almost certainly fail, and you will lose points,
and the training and testing of the machine learning model will absolutely fail. So, please take close
attention.

'''
X_test_ohe = []  #one-hot encoded test dataset

#Here below is your task: one-hot encoding of test set: X_test
# YOUR CODE HERE
X_test_ohe, enc_list, categorical_features = lets_do_one_hot_encoding(X_test ,categorical_features, transform_only=True, encoders=enc_list)

if(len(X_train_ohe.columns) == len(X_test_ohe.columns)):
    print("Same number of columns i.e",+len(X_train_ohe.columns))



Same number of columns i.e 45


In [12]:
assert len(X_train_ohe.columns)== len(X_test_ohe.columns) and len(X_train_ohe.columns) == sum([1 for i, j in zip(X_train_ohe.columns, X_test_ohe.columns) if i == j])
assert (X_train_ohe.shape, X_test_ohe.shape)==((76050, 45), (25350, 45))

TASK 5:
Given the X_train_ohe (Onehot encoded Pandas Dataframe from Task 4), check if there are missing values, and if yes, count how many, and impute the missing values with corresponding mean values.
Finally, print the counting result as a Pandas dataframe named "missing_counts" having 2 columns {variable_name,num_of_missing_values). Please make sure that the result lists all the input variables in the given dataset.
Now, impute the missing values by mean of the respective variable and save the revised dataframe as X_train_ohe_imputed.

In [13]:
missing_counts = pd.DataFrame()
X_train_ohe_imputed = X_train_ohe

#Your task below. 
# i) Please prepare the missing_counts dataframe accordingly.
# ii) impute missing value in a variable with corresponding mean of the variable.
# YOUR CODE HERE

missing_counts = pd.DataFrame(X_train_ohe.isnull().sum()).rename_axis("variable_name", axis = 1)
missing_counts.rename(columns = {0: "num_of_missing_values"}, inplace = True)

print(missing_counts)
print(X_train_ohe.columns[X_train_ohe.isnull().any()], X_train_ohe.isnull().any(axis = 1).sum())


X_train_ohe_imputed = X_train_ohe.fillna(X_train_ohe.mean())
print("After imputing the missing values")
print(X_train_ohe_imputed.columns[X_train_ohe_imputed.isnull().any()], X_train_ohe_imputed.isnull().any(axis = 1).sum())



variable_name  num_of_missing_values
SEX                                0
MARITAL                            0
FAGE                               0
GAINED                             1
VISITS                             0
MAGE                               0
FEDUC                              1
MEDUC                              0
TOTALP                             0
BDEAD                              0
TERMS                              0
LOUTCOME                           0
WEEKS                              1
RACEMOM                            0
RACEDAD                            0
CIGNUM                             1
DRINKNUM                           0
ANEMIA                             0
CARDIAC                            0
ACLUNG                             0
DIABETES                           0
HERPES                             0
HYDRAM                             1
HEMOGLOB                           0
HYPERCH                            0
HYPERPR                            0
E

In [14]:
assert sum(missing_counts['num_of_missing_values'])==5
assert np.isnan(X_train_ohe.loc[[1748]][['FEDUC']].iloc[0,0])==True
assert np.isnan(X_train_ohe_imputed.loc[[1748]][['FEDUC']].iloc[0,0])==False

TASK 6:
Given a X_train_ohe_imputed (Pandas dataframe from Task 5) where all the categorical variables are already replaced with numeric values, print a list of top 20 highly correlated variables with respect to the target variable, and save the result as a Pandas dataframe named top20_df with 2 columns {variable,corr_score}.
Here, the corr_score between a variable x and the target variable y needs to be computed using the Pearson Correlation Coefficient (PCC). Please note, PCC ranges between -1 to +1. PCC score 0 means no correlation, while value towards +1 and -1 represent positive and negative correlations respectively. For instance, PCC=0.8 and PCC=-0.8 tell similar strength positive and negative correlations between the two subject variables.
Please do not include BWEIGHT in the top20_df list of top 20 correlated variable list.

In [15]:
top20_df = pd.DataFrame() #You need to save the top-20 most correlated variables with respect to BWEIGHT target

# Your task below is:
# i) on the imputed dataset you got from task 5, find the top-20 most-correlated variables
# with respect to the target variable.

# YOUR CODE HERE

corr_score = X_train_ohe_imputed.corrwith(y_train, axis=0, drop=True, method='pearson')


corr_score = corr_score.rename_axis("variable").reset_index()
corr_score.rename(columns={ corr_score.columns[1]: "Corr Score" }, inplace = True)


top20_df = corr_score.sort_values(by = "Corr Score", ascending = False, key = abs).iloc[:20]
print(top20_df)
print(top20_df.shape)

   variable  Corr Score
12    WEEKS    0.565254
3    GAINED    0.174466
4    VISITS    0.129247
25  HYPERPR   -0.111375
1   MARITAL   -0.106297
0       SEX   -0.091260
15   CIGNUM   -0.086711
14  RACEDAD   -0.084284
13  RACEMOM   -0.078811
29  PRETERM   -0.075237
27   CERVIX   -0.068751
5      MAGE    0.068550
28  PINFANT    0.065963
26   ECLAMP   -0.065429
7     MEDUC    0.055249
2      FAGE    0.051981
6     FEDUC    0.051537
22   HYDRAM   -0.050019
24  HYPERCH   -0.047094
32  UTERINE   -0.040339
(20, 2)


In [16]:
assert top20_df.shape==(20,2)

TASK 7:
Given the X_train_ohe_imputed (as Pandas dataframe from task 5) and and top20_df (as Pandas Dataframe from Task 6) having 2 columns {variable_name,corr_score} similar to the one you computed in Task 6:

Please save as X_train_t20 keeping only the columns listed in the top20_df dataframe.
Repeat the process for X_test_ohe (obtained from task 4), and save it as X_test_t20.

In [17]:
#Your task below is to slice training and test dataset to retain only the top-20 most-correlated variables
# with respect to the target variable

X_train_t20 = pd.DataFrame()
X_test_t20 = pd.DataFrame()

# YOUR CODE HERE

column_list = [i for i in top20_df["variable"].values]
X_train_t20 = X_train_ohe_imputed.loc[ :,column_list]
print(X_train_t20.shape)

# process for X_test_ohe

missing_values = pd.DataFrame(X_test_ohe.isnull().sum()).rename_axis("variable_name", axis = 1)
missing_values.rename(columns = {0: "num_of_missing_values"}, inplace = True)
#print(missing_values)
print(X_test_ohe.columns[X_test_ohe.isnull().any()], X_test_ohe.isnull().any(axis = 1).sum())

X_test_ohe_imputed = X_test_ohe.copy()

corr_score_test = X_test_ohe_imputed.corrwith(y_test, axis=0, drop=True, method='pearson')
corr_score_test = corr_score_test.rename_axis("variable").reset_index()
corr_score_test.rename(columns={ corr_score_test.columns[1]: "Corr Score" }, inplace = True)
top20_test_df = corr_score_test.sort_values(by = "Corr Score", ascending = False, key = abs).iloc[:20]
print(top20_test_df)

X_test_t20 = X_test_ohe_imputed.loc[ :,column_list]
print(X_test_t20.shape)




(76050, 20)
Index([], dtype='object') 0
   variable  Corr Score
12    WEEKS    0.565732
3    GAINED    0.169653
4    VISITS    0.130641
1   MARITAL   -0.109535
15   CIGNUM   -0.102883
0       SEX   -0.099990
25  HYPERPR   -0.099812
14  RACEDAD   -0.083358
13  RACEMOM   -0.083194
26   ECLAMP   -0.075661
28  PINFANT    0.070742
5      MAGE    0.068241
29  PRETERM   -0.066039
27   CERVIX   -0.061248
7     MEDUC    0.057872
6     FEDUC    0.056074
2      FAGE    0.049824
22   HYDRAM   -0.049789
32  UTERINE   -0.049738
9     BDEAD   -0.044062
(25350, 20)


In [18]:
assert (X_train_t20.shape,X_test_t20.shape)==((76050, 20), (25350, 20))
assert len(X_train_t20.columns)== len(X_test_t20.columns) and len(X_train_t20.columns) == sum([1 for i, j in zip(X_train_t20.columns, X_test_t20.columns) if i == j])

TASK 8:
Apply min-max scaling on the training dataset (X_train_t20 obtained from Task 7). Save the result as X_train_scaled_mm.
Then scale the test dataset (X_test_t20 obtained from Task 7) based on the metrics you obtain when you scale the training dataset. Save the result as X_test_scaled_mm.
PLEASE DO NOT SCALE y_train and y_test.

In [19]:
X_train_scaled_mm = np.array(X_train_t20)
X_test_scaled_mm = np.array(X_test_t20)

#Your task below:
# YOUR CODE HERE

X_train_scaled_mm = MinMaxScaler().fit_transform(X_train_t20)
print(X_train_scaled_mm)
print(X_train_scaled_mm.shape)

X_test_scaled_mm = MinMaxScaler().fit_transform(X_test_t20)
print(X_test_scaled_mm)
print(X_test_scaled_mm.shape)

# X_train_scaled_mm = (X_train_scaled_mm - X_train_t20.min().min())/(X_train_t20.max().max() - X_train_t20.min().min())
# print(X_train_scaled_mm)
# print(X_train_scaled_mm.shape)

# X_test_scaled_mm = (X_test_scaled_mm - X_test_t20.min().min())/(X_test_t20.max().max() - X_test_t20.min().min())
# print(X_test_scaled_mm)
# print(X_test_scaled_mm.shape)


[[0.74074074 0.51020408 0.32653061 ... 0.         0.         0.        ]
 [0.85185185 0.51020408 0.26530612 ... 0.         0.         0.        ]
 [0.81481481 0.3877551  0.2244898  ... 0.         0.         0.        ]
 ...
 [0.77777778 0.30612245 0.20408163 ... 0.         0.         0.        ]
 [0.74074074 0.23469388 0.26530612 ... 0.         0.         0.        ]
 [0.77777778 0.24489796 0.20408163 ... 0.         0.         0.        ]]
(76050, 20)
[[0.77777778 0.33673469 0.32653061 ... 0.         0.         0.        ]
 [0.77777778 0.20408163 0.20408163 ... 0.         1.         0.        ]
 [0.74074074 0.34693878 0.24489796 ... 0.         0.         0.        ]
 ...
 [0.85185185 0.28571429 0.06122449 ... 0.         0.         0.        ]
 [0.66666667 0.29591837 0.20408163 ... 0.         0.         0.        ]
 [0.74074074 0.30612245 0.24489796 ... 0.         0.         0.        ]]
(25350, 20)


In [20]:
assert abs(sum(np.array([min(X_train_scaled_mm[:,0]),max(X_train_scaled_mm[:,0]),min(X_test_scaled_mm[:,0]),max(X_test_scaled_mm[:,0])])-np.array([0.0,1.0,0.0,1.0])))<1e-4


TASK 9:
Apply standardization (i.e., normalization) scaling on the training dataset (X_train_t20 obtained from Task 7). Save the result as X_train_scaled_std.
Then scale the test dataset (X_test_t20 obtained from Task 7) based on the metrics you obtain when you scale the training dataset. Save the result as X_test_scaled_std.
PLEASE DO NOT SCALE y_train and y_test.

In [21]:
X_train_scaled_std = np.array(X_train_t20)
X_test_scaled_std = np.array(X_test_t20)

#Your task below:
# YOUR CODE HERE

X_train_scaled_std = StandardScaler().fit_transform(X_train_t20)
print(X_train_scaled_std)

X_test_scaled_std = StandardScaler().fit_transform(X_test_t20)
print(X_test_scaled_std)


# X_train_scaled_std =(X_train_scaled_std-X_train_scaled_std.mean())/X_train_scaled_std.std()
# print(X_train_scaled_std)

# X_test_scaled_std =(X_test_scaled_std-X_test_scaled_std.mean())/X_test_scaled_std.std()
# print(X_test_scaled_std)



[[-0.29311281  1.45317334  0.95158568 ... -0.12308901 -0.1183216
  -0.0582307 ]
 [ 0.90339477  1.45317334  0.14891977 ... -0.12308901 -0.1183216
  -0.0582307 ]
 [ 0.50455891  0.5703685  -0.38619084 ... -0.12308901 -0.1183216
  -0.0582307 ]
 ...
 [ 0.10572305 -0.01816806 -0.65374615 ... -0.12308901 -0.1183216
  -0.0582307 ]
 [-0.29311281 -0.53313756  0.14891977 ... -0.12308901 -0.1183216
  -0.0582307 ]
 [ 0.10572305 -0.45957049 -0.65374615 ... -0.12308901 -0.1183216
  -0.0582307 ]]
[[ 0.10507939  0.19067582  0.96752115 ... -0.12069863 -0.11763412
  -0.06035238]
 [ 0.10507939 -0.75951723 -0.65291564 ... -0.12069863  8.500935
  -0.06035238]
 [-0.29518546  0.26376759 -0.11277004 ... -0.12069863 -0.11763412
  -0.06035238]
 ...
 [ 0.90560909 -0.17478305 -2.54342522 ... -0.12069863 -0.11763412
  -0.06035238]
 [-1.09571515 -0.10169127 -0.65291564 ... -0.12069863 -0.11763412
  -0.06035238]
 [-0.29518546 -0.0285995  -0.11277004 ... -0.12069863 -0.11763412
  -0.06035238]]


In [22]:
for i in np.arange(20):
    assert abs(X_train_scaled_std[:,i].mean()-0.0)<1e-4 and abs(X_train_scaled_std[:,i].std()-1.0)<1e-4


TASK 10:
Given the (X_train_scaled_std, y_train) pairs denoting input matrix and output vector respectively: complete the three function definitions and demonstrate the functionalities of each by calling them with appropriate arguments as instructed below:

linear_regression_closed_form_training : It fits a linear regression model using the closed-form solution to obtain the coefficients, beta's, as a numpy array of m+1 values (Please recall class lecture), where m is the number of variables kept in X_train (the first argument to the function). Please measure the cpu_time needed during the training step. cpu_time is not equal to the wall_time. So, use time.perf_counter() for an accurate measurement. Documentation on this function can be found here: https://docs.python.org/3/library/time.html . Finally, the function returns betas (i.e., the m+1 beta values) and the cpu_time.

linear_regression_closed_form_predict: It takes a list of m+1 beta values (i.e., betas returned from the corresponding training function, and X_test (containing test samples each having m input variables). Now, using the provided beta values, predict each of the test samples provided, and let's name your prediction "y_pred". Return y_pred from the function.

RMSLE: It takes two lists: y_test, y_pred, where the first list represents ground truth (i.e., actual) target values for the given samples, and the second list represents a corresponding predicted values for exactly same number of samples in y_test. Compute and return the Root Mean Squared Logarithmic Error (RMSLE) of the prediction.

PLEASE DO NOT USE ANY LIBRARY FUNCTION THAT DOES THE LINEAR REGRESSION or RMSLE calculation.

Now, call linear_regression_closed_form_training() function providing X_train_scaled_std, y_train obtained from Task 9, and save the returned results as betas_closed_form,cpu_time_closed_form.

Print betas_closed_form, cpu_time_closed_form

Call linear_regression_closed_form_predict() function providing betas_closed_form,X_test_scaled_std obtained in Task 9. Save the returned result as y_pred.

Call RMSLE() function providing y_test and y_pred. Save returned result as rmsle_closed_form.

Print rmsle_closed_form.

In [23]:
def linear_regression_closed_form_training(X_train, y_train):
    betas = []
    cpu_time = 0

    #Your work here
    # YOUR CODE HERE

    start_time = perf_counter()
    X_train = pd.DataFrame(X_train)
    X_train['bias'] = 1 
    beta = np.dot((np.linalg.inv(np.dot(X_train.T, X_train))), np.dot(X_train.T,y_train))
    betas.append(beta)
    
    end_time = perf_counter()
    cpu_time = end_time - start_time
    
    
    return betas,cpu_time

def linear_regression_closed_form_predict(betas, X_test):
    y_pred = []
    #your work below
    # YOUR CODE HERE
    X_test = pd.DataFrame(X_test)
    X_test['bias'] = 1
    
    y_pred = np.dot(X_test, betas)
    return y_pred

def RMSLE(y_test, y_pred, verbose=False):
    rmsle_val = 0
    #Your work below
    # YOUR CODE HERE

    y_test = np.array(y_test)
    y_pred = np.array(y_pred)
    y_pred = abs(y_pred)
    rmsle_val = np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_test))))
    return rmsle_val

  

In [24]:
betas_closed_form = []
cpu_time_closed_form = 0
#Your task here, call appropriate function to get these two
# YOUR CODE HERE
betas_closed_form, cpu_time_closed_form  = linear_regression_closed_form_training(X_train_scaled_std, y_train)
print("Linear regression CPU Time:",cpu_time_closed_form)
betas_closed_form = [i for blist in betas_closed_form for i in blist]
betas_closed_form = np.array(betas_closed_form)
print("Beta Values are:\n",betas_closed_form)


Linear regression CPU Time: 0.04536215799999965
Beta Values are:
 [ 0.71519322  0.16451626  0.03403515 -0.08264272 -0.07634788 -0.12961155
 -0.09253333 -0.05847441 -0.04475425 -0.03880411 -0.02739149  0.10283456
  0.0723325  -0.03912292 -0.01662473  0.00921883 -0.01519333 -0.04398384
 -0.02638443 -0.01920356  7.25699786]


In [25]:
y_pred = []
#Your task here. call appropriate function to get get y prediction for the supplied test dataset scaled
# YOUR CODE HERE
y_pred = linear_regression_closed_form_predict(betas_closed_form, X_test_scaled_std)
#y_pred = [i for blist in y_pred for i in blist]
#y_pred = pd.DataFrame(y_pred)
y_pred


array([7.38676756, 6.76686327, 6.98815652, ..., 7.93817651, 6.61901319,
       7.43494415])

In [26]:
rmsle_closed_form = 0
#Your task below. Call the function to compute RMSLE score of the y predictions
# YOUR CODE HERE
rmsle_closed_form = RMSLE(y_test, y_pred)
print(rmsle_closed_form)

0.13893479990000063


In [27]:
assert abs(RMSLE([1,2,3,4,5],[1,1,1,1,1])-0.733674673672524)<1e-4
assert abs(RMSLE([100, 200, 300, 400, 500], [90, 190, 290, 390, 490])-0.05596497907273607)<1e-4
assert (betas_closed_form.shape[0],y_pred.shape[0])==(21,25350)

TASK 11:
Given the (X_train_scaled_std, y_train) pairs denoting input matrix and output vector respectively: complete the three function definitions and demonstrate the functionalities of each by calling them with appropriate arguments as instructed below:

linear_regression_gd_batch_training : It fits a linear regression model using the batch gradient descent algorithm to obtain the coefficients, beta's, as a numpy array of m+1 values, where m is the number of variables kept in X_train (the first argument to the function). Make sure you compute average of gradients in the batch. Please use the alpha (i.e, the learning rate) and nEpoch (number of epochs) parameters in your implementation of the gradient descent algorithm. Please measure the cpu_time needed during the training step. cpu_time is not equal to the wall_time. So, use time.perf_counter() for an accurate measurement. Documentation on this function can be found here: https://docs.python.org/3/library/time.html . Finally, the function returns betas (i.e., the m+1 beta values) and the cpu_time.

linear_regression_gd_batch_predict: It takes a list of m+1 beta values (i.e., betas returned from the corresponding training function, and X_test (containing test samples each having m input variables). Now, using the provided beta values, predict each of the test samples provided, and let's name your prediction "y_pred". Return y_pred from the function.

PLEASE DO NOT USE ANY LIBRARY FUNCTION THAT DOES THE LINEAR REGRESSION.

Now, call linear_regression_gd_batch_training() function providing X_train_scaled_std, y_train obtained from Task 9, and alpha=0.01,nEpoch=1000, and save the returned results as betas_batch,cpu_time_batch.

Print betas_batch, cpu_time_batch

Call linear_regression_gd_batch_predict() function providing betas_batch,X_test_scaled_std obtained in Task 9. Save the returned result as y_pred.

Call RMSLE() function providing y_test and y_pred. Save returned result as rmsle_batch.

Print rmsle_batch.

In [28]:
def linear_regression_gd_batch_training(X_train,y_train, alpha, nEpoch):
    random.seed(554433)
    betas = []
    cpu_time = 0
    epsilon = 1e-6
    #Your task below
    # YOUR CODE HERE
    
    startTime = perf_counter()
    X_train = pd.DataFrame(X_train)
    X_train['bias'] = 1
    
    feature_count = X_train.shape[1]
    betas = np.ones(shape = feature_count)
    
    m = X_train.shape[0]
    
    for i in range(nEpoch):
        y_predict = np.dot(betas, X_train.T)
        grad = -(2/m) * (np.dot(X_train.T,(y_train - y_predict)))
        betas = betas - alpha * grad
    
    endTime = perf_counter()
    cpu_time = endTime - startTime
    
    return betas, cpu_time

def linear_regression_gd_batch_predict(betas,X_test):
    y_pred = []
    
    #Your task below
    # YOUR CODE HERE
    X_test = pd.DataFrame(X_test)
    X_test['bias'] = 1
    y_pred = (np.dot(X_test, betas))
    return y_pred



In [29]:
betas_batch = []
cpu_time_batch = 0

#your task below
# YOUR CODE HERE
alpha=0.01
nEpoch=1000

betas_batch,cpu_time_batch = linear_regression_gd_batch_training(X_train_scaled_std, y_train, alpha, nEpoch)

print("Linear Regression Batch time =", cpu_time_batch)
betas_batch = pd.DataFrame(betas_batch)
print("Beta Values are:\n",betas_batch)


Linear Regression Batch time = 7.305574580000002
Beta Values are:
            0
0   0.715199
1   0.164523
2   0.034034
3  -0.082642
4  -0.076325
5  -0.129610
6  -0.092523
7  -0.058669
8  -0.044552
9  -0.038804
10 -0.027392
11  0.103042
12  0.072332
13 -0.039122
14 -0.016642
15  0.009027
16 -0.015195
17 -0.043983
18 -0.026386
19 -0.019204
20  7.256998


In [30]:
y_pred = []

#your task below
# YOUR CODE HERE
y_pred = linear_regression_gd_batch_predict(betas_batch, X_test_scaled_std)
y_pred = [i for blist in y_pred for i in blist]
y_pred = pd.DataFrame(y_pred)
y_pred

,0
0,7.386779
1,6.766822
2,6.988124
3,7.201842
4,7.285637
...,...
25345,7.195200
25346,7.421599
25347,7.938386
25348,6.620237


In [31]:
rmsle_batch = 0
# YOUR CODE HERE
rmsle_batch = RMSLE(y_test, y_pred)
print(rmsle_batch)


0.2248902523028116


## TASK 12:
Given the (X_train_scaled_std, y_train) pairs denoting input matrix and output vector respectively: complete the three function definitions and demonstrate the functionalities of each by calling them with appropriate arguments as instructed below:
* **linear_regression_gd_stochastic_training** : It fits a linear regression model using the stochastic gradient descent algorithm to obtain the coefficients, beta's, as a numpy array of m+1 values, where *m* is the number of variables kept in X_train (the first argument to the function). Please use the alpha (i.e, the learning rate), nEpoch (number of epochs), nIteration (number of iterations) parameters in your implementation of the gradient descent algorithm. Please measure the cpu_time needed during the training step. cpu_time is not equal to the wall_time. So, use time.perf_counter() for an accurate measurement. Documentation on this function can be found here: https://docs.python.org/3/library/time.html . Finally, the function returns betas (i.e., the m+1 beta values) and the  cpu_time.
* **linear_regression_gd_stochastic_predict**: It takes a list of m+1 beta values (i.e., betas returned from the corresponding training function, and X_test (containing test samples each having *m* input variables). Now, using the provided beta values, predict each of the test samples provided, and let's name your prediction "y_pred". Return y_pred from the function. 

* PLEASE DO NOT USE ANY LIBRARY FUNCTION THAT DOES THE LINEAR REGRESSION.
* Now, call linear_regression_gd_stochastic_training() function providing X_train_scaled_std, y_train obtained from Task 9, and alpha=0.001,nEpoch=100, nIteration=10000 , and save the returned results as betas_stochastic,cpu_time_stochastic.
* Print betas_stochastic, cpu_time_stochastic
* Call linear_regression_gd_stochastic_predict() function providing betas_stochastic,X_test_scaled_std obtained in Task 9. Save the returned result as y_pred.
* Call RMSLE() function providing y_test and y_pred. Save returned result as rmsle_stochastic.
* Print rmsle_stochastic.

In [32]:
def linear_regression_gd_stochastic_training(X_train,y_train,alpha,nEpoch,nIteration):
    random.seed(554433)
    betas = []
    cpu_time = 0
    epsilon = 1e-6

    ## YOUR CODE HERE
    # YOUR CODE HERE
    
    startTime = perf_counter()
    X_train = pd.DataFrame(X_train)
    X_train['bias'] = 1
    m = X_train.shape[0]
    
    feature_count = X_train.shape[1]
    betas = np.ones(shape = feature_count)
    
    for i in range(nEpoch):
        temp = random.randint(0,m-1)
        temp_x = X_train.iloc[temp]
        temp_y = y_train.iloc[temp]
        y_predict = np.dot(betas, temp_x.T) 
        grad = -(2/m) * (np.dot(temp_x.T,(temp_y - y_predict)))
        betas = betas - alpha * grad
        
    endTime = perf_counter()
    cpu_time = endTime - startTime
    
    
    
    start_time = perf_counter()
    #n, m = X_train.shape
    for i in range(nIteration):
        start_time = perf_counter()
        #n, m = X_train.shape
        #betas with shape (m+1,1)
        m,n = X_train.shape
        #beta = np.random.rand(n) #X_train_scaled_std, y_train
        betas_stochastic = inv(X_train_scaled_std.T.dot(X_train_scaled_std)).dot(X_train_scaled_std.T).dot(y_train)
    for _ in range(nIteration):
        x = "-".join(str(n) for n in range(nEpoch))
        end = perf_counter()
    #rint('Perf Counter= ', end-start_time)
    cpu_time = end-start_time
    
    return betas, cpu_time

def linear_regression_gd_stochastic_predict(betas, X_test):
    y_pred = []
    
    #your code below
    # YOUR CODE HERE
    X_test = pd.DataFrame(X_test)
    X_test['bias'] = 1
    y_pred = np.dot(X_test, betas)
    
    return y_pred
    



In [33]:
betas_stochastic = []
cpu_time_stochastic = 0
#your code below
# YOUR CODE HERE
alpha=0.001
nEpoch=100
nIteration=1000
betas_stochastic, cpu_time_stochastic = linear_regression_gd_stochastic_training(X_train_scaled_std,y_train,alpha,nEpoch,nIteration)
print("CPU Time:", cpu_time_stochastic)
betas_stochastic = pd.DataFrame(betas_stochastic)
print("Betas:")
print(betas_stochastic)


CPU Time: 0.028914396000004672
Betas:
           0
0   0.999999
1   0.999998
2   0.999997
3   0.999997
4   1.000003
5   0.999997
6   0.999998
7   0.999997
8   0.999997
9   0.999994
10  0.999999
11  0.999994
12  1.000000
13  0.999999
14  0.999993
15  0.999994
16  0.999994
17  0.999998
18  0.999997
19  0.999999
20  1.000016


In [34]:
y_pred = []
#your code below
# YOUR CODE HERE
y_pred = linear_regression_gd_stochastic_predict(betas_stochastic, X_test_scaled_std)
print("Y Prediction:")
print(y_pred)

Y Prediction:
[[ 3.38063447]
 [ 6.76612078]
 [-1.55878885]
 ...
 [-3.13178069]
 [ 6.91674302]
 [ 2.0585277 ]]


In [35]:
rmsle_stochastic = 0
#your code below
# YOUR CODE HERE
rmsle_stochastic = RMSLE(y_test, y_pred)
print(rmsle_batch)

0.2248902523028116


Task 13:
Given the (X_train_scaled_std, y_train) pairs denoting input matrix and output vector respectively: complete the three function definitions and demonstrate the functionalities of each by calling them with appropriate arguments as instructed below:

linear_regression_gd_minibatch_training : It fits a linear regression model using the minibatch gradient descent algorithm to obtain the coefficients, beta's, as a numpy array of m+1 values, where m is the number of variables kept in X_train (the first argument to the function). Please use the alpha (i.e, the learning rate), nEpoch (number of epochs), nIteration (number of iterations), and batch_size parameters in your implementation of the gradient descent algorithm. Please measure the cpu_time needed during the training step. cpu_time is not equal to the wall_time. So, use time.perf_counter() for an accurate measurement. Documentation on this function can be found here: https://docs.python.org/3/library/time.html . Finally, the function returns betas (i.e., the m+1 beta values) and the cpu_time.

linear_regression_gd_minibatch_predict: It takes a list of m+1 beta values (i.e., betas returned from the corresponding training function, and X_test (containing test samples each having m input variables). Now, using the provided beta values, predict each of the test samples provided, and let's name your prediction "y_pred". Return y_pred from the function.

PLEASE DO NOT USE ANY LIBRARY FUNCTION THAT DOES THE LINEAR REGRESSION.

Now, call linear_regression_gd_minibatch_training() function providing X_train_scaled_std, y_train obtained from Task 9, and alpha=0.01,nEpoch=50, nIteration=1000,batch_size=32, and save the returned results as betas_minibatch,cpu_time_minibatch.

Print betas_minibatch, cpu_time_minibatch

Call linear_regression_gd_minibatch_predict() function providing betas_minibatch,X_test_scaled_std obtained in Task 9. Save the returned result as y_pred.

Call RMSLE() function providing y_test and y_pred. Save returned result as rmsle_minibatch.

Print rmsle_minibatch.

In [36]:
def linear_regression_gd_minibatch_training(X_train,y_train,alpha,nEpoch, nIteration, batch_size):
    random.seed(554433)
    cpu_time = 0
    epsilon = 1e-6
    betas = []
    ## your code below
    # YOUR CODE HERE
    
    startTime = perf_counter()
    X_train = pd.DataFrame(X_train)
    m = X_train.shape[0]
    
    X_train['bias'] = 1
    feature_count = X_train.shape[1]
    
    #betas must be initialized with any value. This function will obtain the optimum beta values by iterating through nEpoch
    betas = np.ones(shape = feature_count)
    
    if(batch_size >= m):
        batch_size = m
    
    number_of_batches = int(m/batch_size)
    
    for i in range (nEpoch):
        temp = np.random.permutation(m)
        
        temp_x = X_train.iloc[temp]
        temp_y = y_train.iloc[temp]
        
        for j in range (0, m, batch_size):
            batch_x = temp_x[j:j+batch_size]
            batch_y = temp_y[j:j+batch_size]
            y_predict = np.dot(betas, batch_x.T)
            grad = -(2/m) * (np.dot(batch_x.T,(batch_y - y_predict)))
            betas = betas - alpha * grad
        
    endTime = perf_counter()
    cpu_time = endTime - startTime
    
    return betas,cpu_time

def linear_regression_gd_minibatch_predict(betas,X_test):
    #your solution below
    y_pred = []
    
    # YOUR CODE HERE
    X_test = pd.DataFrame(X_test)
    X_test['bias'] = 1
    y_pred = np.dot(X_test, betas)
    
    return y_pred


In [37]:
betas_minibatch = []
cpu_time_minibatch = 0
#your task below

# YOUR CODE HERE
alpha=0.01
nEpoch=50
nIteration=1000
batch_size=32

betas_minibatch,cpu_time_minibatch = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
print("CPU Time:",cpu_time_minibatch)
print("Betas:")
betas_minibatch = pd.DataFrame(betas_minibatch)
print(betas_minibatch)


/var/folders/yy/pk8ymktn2997l12961d0jwnh0000gn/T/ipykernel_88703/2033779854.py:32: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  batch_y = temp_y[j:j+batch_size]


CPU Time: 99.46170933399998
Betas:
           0
0   0.949977
1   0.424020
2   0.303324
3   0.292158
4   0.651022
5   0.293780
6   0.358615
7   0.110519
8   0.129453
9   0.290265
10  0.318634
11  0.266705
12  0.413039
13  0.314207
14  0.160562
15  0.214740
16  0.155814
17  0.312966
18  0.317729
19  0.345461
20  4.955185


In [38]:
y_pred = []
#your ans below
# YOUR CODE HERE
y_pred = linear_regression_gd_minibatch_predict(betas_minibatch,X_test_scaled_std)
print("Y Prediction:")
print(y_pred)


Y Prediction:
[[5.29087717]
 [6.46789182]
 [4.60066486]
 ...
 [4.38665844]
 [4.407233  ]
 [4.48286003]]


In [39]:
rmsle_minibatch = 0
#your ans below
# YOUR CODE HERE
rmsle_minibatch = RMSLE(y_test, y_pred)
print(rmsle_minibatch)

0.48803349517674804


Task 14:
Given the 4 sets of results from the 4 experiments (from Tasks 10, 11, 12, 13) with closed form solution, batch gradient descent, stochastic gradient descent and mini-batch gradient descent, assign a string from the set {"closed-form", "batch-GD", "stochastic-GD", "minibatch-GD"} that demonstrated the best predictive performance in terms of RMSE to a variable named best_name.

In [40]:
names = ["closed-form", "batch-GD", "stochastic-GD", "minibatch-GD"]
rmsle_vals = [rmsle_closed_form, rmsle_batch, rmsle_stochastic, rmsle_minibatch]

best_name = 'xxxx'
# YOUR CODE HERE

minimum = rmsle_vals[0]
best_name = names[0]
 
for i in range(1, len(rmsle_vals)):
   
    if rmsle_vals[i] < minimum:
        minimum = rmsle_vals[i]
        best_name = names[i]
print("\nLeast RMSLE is:", minimum)
print("\nBest name is :", best_name) 




Least RMSLE is: 0.13893479990000063

Best name is : closed-form


Task 15:
Given the 4 sets of results from the 4 experiments (from Tasks 10, 11, 12, 13) with closed form solution, batch gradient descent, stochastic gradient descent and mini-batch gradient descent, assign a string from the set {"closed-form", "batch-GD", "stochastic-GD", "minibatch-GD"} that demonstrated the least training cpu time to a variable called best_name.

In [41]:
names = ["closed-form", "batch-GD", "stochastic-GD", "minibatch-GD"]
cpu_times = [cpu_time_closed_form, cpu_time_batch, cpu_time_stochastic, cpu_time_minibatch]
best_name = 'xxxx'

# YOUR CODE HERE
minimum = cpu_times[0]
best_name = names[0]
 
for i in range(1, len(cpu_times)):
   
    if cpu_times[i] < minimum:
        minimum = cpu_times[i]
        best_name = names[i]
print("\nLeast Training CPU time is:", minimum)
print("\nBest name is :", best_name)




Least Training CPU time is: 0.028914396000004672

Best name is : stochastic-GD


Task 16:
Given the (X_train_scaled_std, y_train) pairs denoting input matrix and output vector respectively,

call your implementation of Task 13: minibatch gradient descent based linear regression for each of these learning rates: {0.001, 0.01, 0.05}, batch sizes: {32, 64, 128, 256}

Please use the nIteration (number of iterations)=100, nEpoch (number of epoch)=50.
For each of the linear regression model, using the computed beta values, predict the test samples provided in the "X_test_scaled_std" argument, and let's name your prediction "y_pred".

Compute Root Mean Squared Logarithmic Error (RMSLE) of your prediction using the RMSLE() function you defined in Task 10.

Finally, assign the learning rate that shows the best test performance to a variable called best_alpha, and also assign as a pandas dataframe named summary with 3 columns: {learning_rate, batch_size, test_RMSLE} containing RMSLE's of the 12 linear regression models, i.e., Cartesian product between the alphas and batch_sizes. Also, assign the best performing batch_size to a variable best_batch_size.

PLEASE DO NOT USE ANY LIBRARY FUNCTION THAT DOES THE LINEAR REGRESSION.

In [42]:
alphas = [0.001, 0.01, 0.05]
batch_sizes = [32, 64, 128, 256]
summary = pd.DataFrame(columns=['learning_rate', 'batch_size', 'test_RMSLE'])
nIteration=1000
nEpoch=50

best_alpha = -1
best_beta = []
best_batch_size = -1

#your code below
# YOUR CODE HERE

#1
alpha = 0.001
batch_size = 32
betas_minibatch_0001_32,cpu_time_minibatch_0001_32 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_0001_32 = linear_regression_gd_minibatch_predict(betas_minibatch_0001_32,X_test_scaled_std)
rmsle_minibatch_0001_32 = RMSLE(y_test, y_pred_0001_32)


#2
alpha = 0.01
batch_size = 64
betas_minibatch_001_64,cpu_time_minibatch_001_64 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_001_64 = linear_regression_gd_minibatch_predict(betas_minibatch_001_64,X_test_scaled_std)
rmsle_minibatch_001_64 = RMSLE(y_test, y_pred_001_64)


#3
alpha = 0.05
batch_size = 128
betas_minibatch_005_128,cpu_time_minibatch_005_128 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_005_128 = linear_regression_gd_minibatch_predict(betas_minibatch_005_128,X_test_scaled_std)
rmsle_minibatch_005_128 = RMSLE(y_test, y_pred_005_128)

#4
alpha = 0.001
batch_size = 256
betas_minibatch_0001_256,cpu_time_minibatch_0001_256 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_0001_256 = linear_regression_gd_minibatch_predict(betas_minibatch_0001_256,X_test_scaled_std)
rmsle_minibatch_0001_256 = RMSLE(y_test, y_pred_0001_256)


#5
alpha = 0.01
batch_size = 32
betas_minibatch_001_32,cpu_time_minibatch_001_32 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_001_32 = linear_regression_gd_minibatch_predict(betas_minibatch_001_32,X_test_scaled_std)
rmsle_minibatch_001_32 = RMSLE(y_test, y_pred_001_32)


#6
alpha = 0.05
batch_size = 64
betas_minibatch_005_64,cpu_time_minibatch_005_64 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_005_64 = linear_regression_gd_minibatch_predict(betas_minibatch_005_64,X_test_scaled_std)
rmsle_minibatch_005_64 = RMSLE(y_test, y_pred_005_64)


#7
alpha = 0.001
batch_size = 128
betas_minibatch_0001_128,cpu_time_minibatch_0001_128 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_0001_128 = linear_regression_gd_minibatch_predict(betas_minibatch_0001_128,X_test_scaled_std)
rmsle_minibatch_0001_128 = RMSLE(y_test, y_pred_0001_128)


#8
alpha = 0.01
batch_size = 256
betas_minibatch_001_256,cpu_time_minibatch_001_256 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_001_256 = linear_regression_gd_minibatch_predict(betas_minibatch_001_256,X_test_scaled_std)
rmsle_minibatch_001_256 = RMSLE(y_test, y_pred_001_256)
 

#9
alpha = 0.05
batch_size = 32
betas_minibatch_005_32,cpu_time_minibatch_005_32 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_005_32 = linear_regression_gd_minibatch_predict(betas_minibatch_005_32,X_test_scaled_std)
rmsle_minibatch_005_32 = RMSLE(y_test, y_pred_005_32)


#10
alpha = 0.001
batch_size = 64
betas_minibatch_0001_64,cpu_time_minibatch_0001_64 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_0001_64 = linear_regression_gd_minibatch_predict(betas_minibatch_0001_64,X_test_scaled_std)
rmsle_minibatch_0001_64 = RMSLE(y_test, y_pred_0001_64)

#11
alpha = 0.01
batch_size = 128
betas_minibatch_001_128,cpu_time_minibatch_001_128 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_001_128 = linear_regression_gd_minibatch_predict(betas_minibatch_001_128,X_test_scaled_std)
rmsle_minibatch_001_128 = RMSLE(y_test, y_pred_001_128)

#12
alpha = 0.05
batch_size = 256
betas_minibatch_005_256,cpu_time_minibatch_001_256 = linear_regression_gd_minibatch_training(X_train_scaled_std, y_train, alpha, nEpoch, nIteration, batch_size)
y_pred_005_256 = linear_regression_gd_minibatch_predict(betas_minibatch_005_256,X_test_scaled_std)
rmsle_minibatch_005_256 = RMSLE(y_test, y_pred_005_256)


summary = pd.DataFrame ({'learning_rate' : [0.001, 0.01, 0.05, 0.001, 0.01, 0.05, 0.001, 0.01, 0.05, 0.001, 0.01, 0.05], 
                         'batch_size' : [32, 64, 128, 256, 32, 64, 128, 256, 32, 64, 128, 256], 
                         'test_RMSLE' :[rmsle_minibatch_0001_32, 
                                        rmsle_minibatch_001_64, 
                                        rmsle_minibatch_005_128,
                                        rmsle_minibatch_0001_256,
                                        rmsle_minibatch_001_32,
                                        rmsle_minibatch_005_64,
                                        rmsle_minibatch_0001_128,
                                        rmsle_minibatch_001_256,
                                        rmsle_minibatch_005_32,
                                        rmsle_minibatch_0001_64,
                                        rmsle_minibatch_001_128,
                                        rmsle_minibatch_005_256]})


print(summary)

best_alpha = summary.iloc[summary['test_RMSLE'].idxmin(),0]
best_batch_size = summary.iloc[summary['test_RMSLE'].idxmin(),1]

print("Best performing learning rate : " , best_alpha)
print("Best performing batch size : " , best_batch_size)






/var/folders/yy/pk8ymktn2997l12961d0jwnh0000gn/T/ipykernel_88703/2033779854.py:32: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  batch_y = temp_y[j:j+batch_size]
/var/folders/yy/pk8ymktn2997l12961d0jwnh0000gn/T/ipykernel_88703/2033779854.py:32: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  batch_y = temp_y[j:j+batch_size]
/var/folders/yy/pk8ymktn2997l12961d0jwnh0000gn/T/ipykernel_88703/2033779854.py:32: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is d

    learning_rate  batch_size  test_RMSLE
0           0.001          32    1.081976
1           0.010          64    0.449556
2           0.050         128    0.138313
3           0.001         256    1.081976
4           0.010          32    0.449558
5           0.050          64    0.138316
6           0.001         128    1.081976
7           0.010         256    0.449541
8           0.050          32    0.138316
9           0.001          64    1.081976
10          0.010         128    0.449550
11          0.050         256    0.138315
Best performing learning rate :  0.05
Best performing batch size :  128


Task 17:
Utilizing the best trained linear regression model (so far), predict the target for each of the samples in the judge_dataset.
I believe you will not forget to do the following before call in the prediction algorithm:
Save the ID values of the judge dataset into ID_judge and drop it from the judge dataframe.
Perform onehot encoding using the same encoder you used to encode X_test (Task 4).
keep only the same top 20 variables as you did in Task 7.
scale the input variables based on the same metrics you used to scale the training dataset (Task 9).
Now, call the prediction function of that model to obtain y_pred.
Prepare a pandas dataframe called result having columns: {ID, BWEIGHT}, where ID will the ID of the judge sample, and BWEIGHT is the corresponding y_pred value from your model prediction.
Save the dataframe as my_submission.csv.
PLEASE DO NOT USE ANY LIBRARY FUNCTION THAT DOES THE LINEAR REGRESSION.

In [43]:
#your task below
#i) store the id in a separate variable and drop it from judge dataset
#ii) use onehot encoder you obtained to encode the judge set
#iii) keep only the top-20 variables in the judge set
#iv) scale using the same scalar you obtained before, and save it to judge_scaled_std variable.
judge_scaled_std = []

# YOUR CODE HERE
# Save the ID values of the judge dataset into ID_judge and drop it from the judge dataframe.
X_judge = judge_dataset
ID_judge = X_judge["ID"]
X_judge = X_judge.drop("ID", axis=1)


# Perform onehot encoding using the same encoder you used to encode X_test (Task 4).
X_judge_ohe, enc_list, categorical_features = lets_do_one_hot_encoding(X_judge ,categorical_features, transform_only=True, encoders=enc_list)

# keep only the same top 20 variables as you did in Task 7.
X_judge_t20 = pd.DataFrame()
for i in range(len(top20_df)):
    try:
        cols = X_judge_ohe.iloc[:,X_judge_ohe.columns.get_loc(top20_df.iloc[i,0])]
        X_judge_t20[top20_df.iloc[i,0]] = cols
    except KeyError:
        pass
print(X_judge_t20)

# scale the input variables based on the same metrics you used to scale the training dataset (Task 9).
judge_scaled_std = StandardScaler().fit_transform(X_judge_t20)
print(judge_scaled_std)




      WEEKS  GAINED  VISITS  HYPERPR  MARITAL  SEX  CIGNUM  RACEDAD  RACEMOM  \
0        36      70      11        0        1    1       0        1        1   
1        41      36      15        0        2    2       0        3        3   
2        39      18      10        0        1    2      20        1        1   
3        39      25      10        0        1    2       0        1        1   
4        38      38      15        1        1    2       0        1        1   
...     ...     ...     ...      ...      ...  ...     ...      ...      ...   
1995     35      17       9        0        1    2       0        1        1   
1996     39      20      10        0        1    1       0        1        1   
1997     40      25      17        0        1    1       0        1        1   
1998     38      29      11        0        1    1       0        1        1   
1999     34      20      13        0        2    1       5        1        1   

      PRETERM  CERVIX  MAGE  PINFANT  E

In [44]:
#your task is to predict using your best regression betas, and save it to `y_pred` variable
y_pred = []

# YOUR CODE HERE

y_pred_batch = linear_regression_gd_batch_predict(betas_batch,judge_scaled_std)
#y_pred_linear = linear_regression_closed_form_predict(betas_closed_form, judge_scaled_std)


In [45]:
#your task below is to prepare a dataframe `result` with columns: {ID,BWEIGHT} as defined in task-17
result = pd.DataFrame()
# YOUR CODE HERE


#result = pd.DataFrame({'ID' : ID_judge, 'BWEIGHT' : y_pred_batch})

y_pred = [item for sublist in y_pred_batch for item in sublist]

result["BWEIGHT"] = y_pred[:2000]


result.insert(0, 'ID', ID_judge)


print(result)
result.to_csv('my_submission.csv',index=False)

        ID   BWEIGHT
0        1  7.143660
1        2  7.516157
2        3  6.525241
3        4  7.148904
4        5  6.766487
...    ...       ...
1995  1996  5.875826
1996  1997  7.370508
1997  1998  7.843530
1998  1999  7.376166
1999  2000  5.726769

[2000 rows x 2 columns]


Task 18:
[Very important] Please look back from tasks 1-17 to see either do hyperparameter tuning, data analysis, model selection to improve your initial submission.

In [47]:
#your task is to re-assign proper values based on the task requirement below.
# YOUR CODE HERE
y_pred = []
#alpha=0.05
#nEpoch=1000
#batch_size=32

y_pred = linear_regression_gd_minibatch_predict(betas_minibatch_005_32,judge_scaled_std)


result = pd.DataFrame({'ID' : ID_judge, 'BWEIGHT' : y_pred})


print(result)

        ID   BWEIGHT
0        1  7.044473
1        2  7.510963
2        3  6.468890
3        4  7.051380
4        5  6.712514
...    ...       ...
1995  1996  5.756322
1996  1997  7.298932
1997  1998  7.761803
1998  1999  7.314196
1999  2000  5.715069

[2000 rows x 2 columns]
